<a href="https://colab.research.google.com/github/lisstatiavila/Cartel-3er-Semestre/blob/main/C%C3%B3digo_para_CARTEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
# Funciones para el modelo M/M/c
def erlang_c(lmbda, mu, c):
    """Calcula la probabilidad de espera (Erlang C)"""
    rho = lmbda / (c * mu)
    if rho >= 1:
        return 1  # sistema inestable
    # Parte 1: sumatoria
    sum_terms = sum((lmbda/mu)**k / math.factorial(k) for k in range(c))
    # Parte 2: término final
    last_term = ((lmbda/mu)**c / math.factorial(c)) * (1 / (1 - rho))
    P0 = 1 / (sum_terms + last_term)
    # Erlang C
    P_wait = last_term * P0
    return P_wait, P0
def mmc_metrics(lmbda, mu, c):
    """Calcula métricas del sistema M/M/c"""
    rho = lmbda / (c * mu)
    if rho >= 1:
        raise ValueError("El sistema es inestable: aumentar c")
    P_wait, P0 = erlang_c(lmbda, mu, c)
    # Lq - clientes promedio en cola
    Lq = (P_wait * rho) / (1 - rho)
    # L - clientes promedio en el sistema
    L = Lq + lmbda / mu
    # Wq - tiempo promedio en cola
    Wq = Lq / lmbda
    # W - tiempo promedio total
    W = Wq + 1/mu
    return {"rho": rho,"P0": P0,"P_wait (Erlang C)": P_wait,"Lq": Lq,"L": L,
        "Wq": Wq,
        "W": W }
# Cálculo automático de servidores recomendados
def recommended_servers(lmbda, mu, rho_obj=0.80):
    """Calcula el número mínimo y recomendado de servidores para un M/M/c"""
    c_min = math.ceil(lmbda / mu)  # condición básica de estabilidad
    c_rec = math.ceil(lmbda / (rho_obj * mu))
    return c_min, c_rec


# EJEMPLO INTERACTIVO
print("=== Modelo M/M/c : FIFO/∞/∞ para transacciones Amazon ===")
# Puedes cambiar estos valores:
lmbda = float(input("Ingresa la tasa λ (llegadas por hora): "))
mu     = float(input("Ingresa la tasa μ (servicios por hora por servidor): "))
c_min, c_rec = recommended_servers(lmbda, mu)
print(f"\nServidores mínimos para estabilidad: c_min = {c_min}")
print(f"Servidores recomendados (ρ <= 0.80): c_rec = {c_rec}")
# Elegimos c recomendado para analizar
c = c_rec
print(f"\nCalculando métricas para c = {c} servidores...\n")
results = mmc_metrics(lmbda, mu, c)
for k, v in results.items():
    print(f"{k}: {v:}")

=== Modelo M/M/c : FIFO/∞/∞ para transacciones Amazon ===
Ingresa la tasa λ (llegadas por hora): 500
Ingresa la tasa μ (servicios por hora por servidor): 300

Servidores mínimos para estabilidad: c_min = 2
Servidores recomendados (ρ <= 0.80): c_rec = 3

Calculando métricas para c = 3 servidores...

rho: 0.5555555555555556
P0: 0.1726618705035971
P_wait (Erlang C): 0.29976019184652286
Lq: 0.3747002398081536
L: 2.0413669064748206
Wq: 0.0007494004796163072
W: 0.004082733812949641
